# MID TERM
## ETL PIPELINE

# From API

In [15]:
import requests
import pandas as pd

# API config
API_KEY = "Y8L1GON8GZFETTF6"
symbol = "IBM"
interval = "5min"
function = "TIME_SERIES_INTRADAY"

# API URL
url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&interval={interval}&apikey={API_KEY}"

# Get response
response = requests.get(url)
data = response.json()

# Check for API errors or rate limit
if "Time Series (5min)" not in data:
    raise Exception(f"Error fetching data: {data.get('Note', data)}")

data

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2025-04-04 19:55:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2025-04-04 19:55:00': {'1. open': '227.4800',
   '2. high': '227.4800',
   '3. low': '226.5300',
   '4. close': '226.5300',
   '5. volume': '28'},
  '2025-04-04 19:50:00': {'1. open': '227.0300',
   '2. high': '227.4460',
   '3. low': '226.1200',
   '4. close': '226.1540',
   '5. volume': '60'},
  '2025-04-04 19:45:00': {'1. open': '227.2400',
   '2. high': '227.3000',
   '3. low': '227.0000',
   '4. close': '227.0300',
   '5. volume': '50'},
  '2025-04-04 19:40:00': {'1. open': '227.4400',
   '2. high': '227.4400',
   '3. low': '227.2400',
   '4. close': '227.2400',
   '5. volume': '2'},
  '2025-04-04 19:35:00': {'1. open': '227.4800',
   '2. high': '227.4800',
   '3. low': '227.4300',
   '4. close': '227.4300',
  

In [23]:
# prompt: SAVE ABOVE data IN stock.json format

import json

# Assuming 'data' is the dictionary you want to save
with open('stock.json', 'w') as f:
    json.dump(data, f, indent=4)


In [18]:
# Define the function BEFORE using it
def convert_intraday_to_dataframe(data, symbol, interval):
    time_series = data[f"Time Series ({interval})"]
    df = pd.DataFrame([
        {
            "symbol": symbol,
            "date": pd.to_datetime(date).tz_localize("US/Eastern").tz_convert("UTC").isoformat(),
            "close": float(values["4. close"]),
            "high": float(values["2. high"]),
            "low": float(values["3. low"]),
            "open": float(values["1. open"]),
            "volume": int(values["5. volume"]),
        }
        for date, values in time_series.items()
    ])
    return df.sort_values("date", ascending=False).reset_index(drop=True)



In [19]:
# Use the function
df_IBM = convert_intraday_to_dataframe(data, symbol, interval)

# Show result
df_IBM.head()

,symbol,date,close,high,low,open,volume
0,IBM,2025-04-04T23:55:00+00:00,226.530,227.480,226.53,227.48,28
1,IBM,2025-04-04T23:50:00+00:00,226.154,227.446,226.12,227.03,60
2,IBM,2025-04-04T23:45:00+00:00,227.030,227.300,227.00,227.24,50
3,IBM,2025-04-04T23:40:00+00:00,227.240,227.440,227.24,227.44,2
4,IBM,2025-04-04T23:35:00+00:00,227.430,227.480,227.43,227.48,16


In [20]:
# Make column name in lower case
df_IBM.columns = df_IBM.columns.str.lower()
df_IBM

,symbol,date,close,high,low,open,volume
0,IBM,2025-04-04T23:55:00+00:00,226.530,227.480,226.5300,227.48,28
1,IBM,2025-04-04T23:50:00+00:00,226.154,227.446,226.1200,227.03,60
2,IBM,2025-04-04T23:45:00+00:00,227.030,227.300,227.0000,227.24,50
3,IBM,2025-04-04T23:40:00+00:00,227.240,227.440,227.2400,227.44,2
4,IBM,2025-04-04T23:35:00+00:00,227.430,227.480,227.4300,227.48,16
...,...,...,...,...,...,...,...
95,IBM,2025-04-04T15:55:00+00:00,233.280,234.420,233.2400,234.12,39294
96,IBM,2025-04-04T15:50:00+00:00,233.980,234.595,233.6800,234.23,52635
97,IBM,2025-04-04T15:45:00+00:00,234.430,234.430,233.0320,233.59,51227
98,IBM,2025-04-04T15:40:00+00:00,233.600,234.130,233.2101,233.60,41911


# FROM DRIVE

In [21]:
# Install required packages (if not already installed)
!pip install gspread gspread_dataframe oauth2client --quiet

In [24]:

# Authenticate and authorize Google Sheets access
from google.colab import auth
auth.authenticate_user()

import gspread
import pandas as pd
from google.auth import default
from gspread_dataframe import get_as_dataframe

# Authorize gspread with your default credentials
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Spreadsheet by name
spreadsheet_name = "Stock Data AAPL"  # Use the name of your spreadsheet
sh = gc.open(spreadsheet_name)

# Select the first worksheet
worksheet = sh.sheet1

# Read the worksheet data into a DataFrame
df = get_as_dataframe(worksheet)

# Display the first few rows of the DataFrame
df.head()

,Date,Close,High,Low,Open,Volume
0,NaN,AAPL,AAPL,AAPL,AAPL,AAPL
1,2024-01-02 00:00:00,184.532074,187.315366,182.792518,186.033057,82488700
2,2024-01-03 00:00:00,183.150391,184.770668,182.335277,183.120571,58414500
3,2024-01-04 00:00:00,180.824341,181.997291,179.800489,181.062899,71983600
4,2024-01-05 00:00:00,180.098694,181.669266,179.094727,180.903872,62303300


In [25]:
# Drop the first row that contains repeated headers
df_AAPL = df.drop(index=0).copy()

# Convert columns to appropriate types
df_AAPL["Date"] = pd.to_datetime(df_AAPL["Date"]).dt.tz_localize("UTC")
df_AAPL["Close"] = pd.to_numeric(df_AAPL["Close"])
df_AAPL["High"] = pd.to_numeric(df_AAPL["High"])
df_AAPL["Low"] = pd.to_numeric(df_AAPL["Low"])
df_AAPL["Open"] = pd.to_numeric(df_AAPL["Open"])
df_AAPL["Volume"] = pd.to_numeric(df_AAPL["Volume"], downcast="integer")

# Add Symbol column
df_AAPL["Symbol"] = "AAPL"

# Reorder columns
df_AAPL = df_AAPL[["Symbol", "Date", "Close", "High", "Low", "Open", "Volume"]]

# Sort by date (optional)
df_AAPL = df_AAPL.sort_values("Date", ascending=False).reset_index(drop=True)

# Show the cleaned DataFrame
df_AAPL.head()

,Symbol,Date,Close,High,Low,Open,Volume
0,AAPL,2024-12-31 00:00:00+00:00,250.144974,253.001833,249.156056,252.162760,39480700
1,AAPL,2024-12-30 00:00:00+00:00,251.923019,253.221595,250.474615,251.952985,35557500
2,AAPL,2024-12-27 00:00:00+00:00,255.309296,258.415896,252.782075,257.546826,42355300
3,AAPL,2024-12-26 00:00:00+00:00,258.735504,259.814335,257.347047,257.906429,27237100
4,AAPL,2024-12-24 00:00:00+00:00,257.916443,257.926411,255.009620,255.209412,23234700


In [26]:
# Make column name in lower case
df_AAPL.columns = df_AAPL.columns.str.lower()
df_AAPL

,symbol,date,close,high,low,open,volume
0,AAPL,2024-12-31 00:00:00+00:00,250.144974,253.001833,249.156056,252.162760,39480700
1,AAPL,2024-12-30 00:00:00+00:00,251.923019,253.221595,250.474615,251.952985,35557500
2,AAPL,2024-12-27 00:00:00+00:00,255.309296,258.415896,252.782075,257.546826,42355300
3,AAPL,2024-12-26 00:00:00+00:00,258.735504,259.814335,257.347047,257.906429,27237100
4,AAPL,2024-12-24 00:00:00+00:00,257.916443,257.926411,255.009620,255.209412,23234700
...,...,...,...,...,...,...,...
247,AAPL,2024-01-08 00:00:00+00:00,184.452560,184.492330,180.416793,181.003268,59144500
248,AAPL,2024-01-05 00:00:00+00:00,180.098694,181.669266,179.094727,180.903872,62303300
249,AAPL,2024-01-04 00:00:00+00:00,180.824341,181.997291,179.800489,181.062899,71983600
250,AAPL,2024-01-03 00:00:00+00:00,183.150391,184.770668,182.335277,183.120571,58414500


# From json

In [ ]:
import json
import pandas as pd

# If you haven't mounted your Google Drive yet, uncomment the following lines:
from google.colab import drive
drive.mount('/content/drive')

# Path to your JSON file in Google Drive
json_file_path = "/content/drive/My Drive/ETL_Pipeline/data/ALIBABA_stock_data.json"  # Update this path

# Open and load the JSON file
with open(json_file_path, "r") as f:
    json_data = json.load(f)
print(json_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'BABA', '3. Last Refreshed': '2025-03-25', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-03-25': {'1. open': '132.9400', '2. high': '136.2450', '3. low': '132.3500', '4. close': '132.7500', '5. volume': '17943385'}, '2025-03-24': {'1. open': '136.3300', '2. high': '136.4300', '3. low': '133.6300', '4. close': '134.4800', '5. volume': '14019013'}, '2025-03-21': {'1. open': '133.9900', '2. high': '136.4700', '3. low': '133.3000', '4. close': '135.1400', '5. volume': '13970265'}, '2025-03-20': {'1. open': '137.1750', '2. high': '138.9900', '3. low': '136.3700', '4. close': '136.9100', '5. volume': '23646782'}, '2025-03-19': {'1. open': '145.0700', '2. high': '145.2000', '3. low': '141.8700', '4. close': '143.2000', '5. vo

In [32]:
# Extract time series data
time_series = json_data["Time Series (Daily)"]

# Convert to DataFrame
df_BABA = pd.DataFrame.from_dict(time_series, orient="index")

# Rename columns for easier access
df_BABA.columns = ["open", "high", "low", "close", "volume"]

# Convert data types
df_BABA = df_BABA.astype({
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": int
})

# Reset the index and format the date
df_BABA = df_BABA.reset_index().rename(columns={"index": "date"})
df_BABA["date"] = pd.to_datetime(df_BABA["date"]).dt.tz_localize("UTC")
df_BABA["date"] = df_BABA["date"].dt.strftime("%Y-%m-%d 00:00:00+00:00")

# Add symbol column
df_BABA["symbol"] = "BABA"

# Sort by date
df_BABA.sort_values("date", inplace=True)

# Preview the DataFrame
df_BABA.head()

,date,open,high,low,close,volume,symbol
99,2024-10-29 00:00:00+00:00,101.450,101.4500,99.530,99.83,13685370,BABA
98,2024-10-30 00:00:00+00:00,98.215,99.8000,97.580,98.70,10179903,BABA
97,2024-10-31 00:00:00+00:00,97.800,98.1000,96.755,97.98,8379347,BABA
96,2024-11-01 00:00:00+00:00,98.220,98.2999,97.420,97.58,8501018,BABA
95,2024-11-04 00:00:00+00:00,98.820,99.8800,98.360,98.40,8681889,BABA


In [33]:
# Make column name in lower case
df_BABA.columns = df_BABA.columns.str.lower()
df_BABA

,date,open,high,low,close,volume,symbol
99,2024-10-29 00:00:00+00:00,101.450,101.4500,99.530,99.83,13685370,BABA
98,2024-10-30 00:00:00+00:00,98.215,99.8000,97.580,98.70,10179903,BABA
97,2024-10-31 00:00:00+00:00,97.800,98.1000,96.755,97.98,8379347,BABA
96,2024-11-01 00:00:00+00:00,98.220,98.2999,97.420,97.58,8501018,BABA
95,2024-11-04 00:00:00+00:00,98.820,99.8800,98.360,98.40,8681889,BABA
...,...,...,...,...,...,...,...
4,2025-03-19 00:00:00+00:00,145.070,145.2000,141.870,143.20,16333817,BABA
3,2025-03-20 00:00:00+00:00,137.175,138.9900,136.370,136.91,23646782,BABA
2,2025-03-21 00:00:00+00:00,133.990,136.4700,133.300,135.14,13970265,BABA
1,2025-03-24 00:00:00+00:00,136.330,136.4300,133.630,134.48,14019013,BABA


# From csv

In [ ]:
import pandas as pd


# Path to your CSV file in Google Drive
csv_file_path = "/content/drive/My Drive/ETL_Pipeline/data/yahoo_data.csv"  # Update this path

# Load the CSV file into a DataFrame
df_csv = pd.read_csv(csv_file_path)

# Display the first few rows
print(df_csv.head())


           Date       Open       High        Low     Close* Adj Close**  \
0  Apr 28, 2023  33,797.43  34,104.56  33,728.40  34,098.16   34,098.16   
1  Apr 27, 2023  33,381.66  33,859.75  33,374.65  33,826.16   33,826.16   
2  Apr 26, 2023  33,596.34  33,645.83  33,235.85  33,301.87   33,301.87   
3  Apr 25, 2023  33,828.34  33,875.49  33,525.39  33,530.83   33,530.83   
4  Apr 24, 2023  33,805.04  33,891.15  33,726.09  33,875.40   33,875.40   

        Volume  
0  354,310,000  
1  343,240,000  
2  321,170,000  
3  297,880,000  
4  252,020,000  


In [35]:
# Drop columns that don't exist in the previous DataFrames (e.g., "Adj Close**")
df_Yahoo = df_csv.drop(columns=["Adj Close**"])

# Convert columns to the correct format
df_Yahoo["Date"] = pd.to_datetime(df_csv["Date"]).dt.strftime("%Y-%m-%d 00:00:00+00:00")
df_Yahoo["Open"] = df_Yahoo["Open"].replace({',': ''}, regex=True).astype(float)
df_Yahoo["High"] = df_Yahoo["High"].replace({',': ''}, regex=True).astype(float)
df_Yahoo["Low"] = df_Yahoo["Low"].replace({',': ''}, regex=True).astype(float)
df_Yahoo["Close*"] = df_Yahoo["Close*"].replace({',': ''}, regex=True).astype(float)

# Handle NaN in Volume: Replace NaNs with 0 before converting to int
df_Yahoo["Volume"] = df_Yahoo["Volume"].replace({',': ''}, regex=True).fillna(0).astype(int)

# Rename the 'Close*' column to match previous DataFrame naming
df_Yahoo.rename(columns={"Close*": "close"}, inplace=True)

# Add Yahoo symbol
df_Yahoo['symbol'] = 'YAHOO'

# Set the index as datetime and sort
df_Yahoo["Date"] = pd.to_datetime(df_Yahoo["Date"])
df_Yahoo.sort_index(inplace=True)

# Preview the cleaned DataFrame
df_Yahoo.head()

,Date,Open,High,Low,close,Volume,symbol
0,2023-04-28 00:00:00+00:00,33797.43,34104.56,33728.40,34098.16,354310000,YAHOO
1,2023-04-27 00:00:00+00:00,33381.66,33859.75,33374.65,33826.16,343240000,YAHOO
2,2023-04-26 00:00:00+00:00,33596.34,33645.83,33235.85,33301.87,321170000,YAHOO
3,2023-04-25 00:00:00+00:00,33828.34,33875.49,33525.39,33530.83,297880000,YAHOO
4,2023-04-24 00:00:00+00:00,33805.04,33891.15,33726.09,33875.40,252020000,YAHOO


In [36]:
# Make column name in lower case
df_Yahoo.columns = df_Yahoo.columns.str.lower()
df_Yahoo

,date,open,high,low,close,volume,symbol
0,2023-04-28 00:00:00+00:00,33797.43,34104.56,33728.40,34098.16,354310000,YAHOO
1,2023-04-27 00:00:00+00:00,33381.66,33859.75,33374.65,33826.16,343240000,YAHOO
2,2023-04-26 00:00:00+00:00,33596.34,33645.83,33235.85,33301.87,321170000,YAHOO
3,2023-04-25 00:00:00+00:00,33828.34,33875.49,33525.39,33530.83,297880000,YAHOO
4,2023-04-24 00:00:00+00:00,33805.04,33891.15,33726.09,33875.40,252020000,YAHOO
...,...,...,...,...,...,...,...
1254,2018-05-04 00:00:00+00:00,23865.22,24333.35,23778.87,24262.51,329480000,YAHOO
1255,2018-05-03 00:00:00+00:00,23836.23,23996.15,23531.31,23930.15,389240000,YAHOO
1256,2018-05-02 00:00:00+00:00,24097.63,24185.52,23886.30,23924.98,385350000,YAHOO
1257,2018-05-01 00:00:00+00:00,24117.29,24117.29,23808.19,24099.05,380070000,YAHOO


# From BigQuery

In [37]:
!pip install google-cloud-bigquery pandas -q
from google.cloud import bigquery
import pandas as pd

In [38]:
from google.colab import auth
auth.authenticate_user()

In [39]:
client = bigquery.Client()

In [40]:
from google.cloud import bigquery

project_id = "smooth-hub-455018-h6"
client = bigquery.Client(project=project_id)

In [41]:
print(f"Using project: {client.project}")

Using project: smooth-hub-455018-h6


In [42]:
!gcloud auth application-default login -q

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=4RKs3QFsGcDoffC0djitIn7r5MsBZL&prompt=consent&token_usage=remote&access_type=offline&code_challenge=z4laqzNwqHMZgYLDX4TVwE9yM6f954bOFZG77OngpjI&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab_5qlnGGdcBumE46Bz_h9Jq2pbVlp7-pKZgPGzGktN7txASvEBlsaJR86TtO_itwhwA_Q

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [43]:
!gcloud config set project smooth-hub-455018-h6


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].


In [44]:
!gcloud auth application-default set-quota-project smooth-hub-455018-h6


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "smooth-hub-455018-h6" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [45]:
from google.cloud import bigquery
# 4. Preview first 10 rows from the `Stock` table
query = """
SELECT *
FROM `smooth-hub-455018-h6.1998.Stock`
"""
df_VOD = client.query(query).to_dataframe()
df_VOD.head()

,Date,Close,High,Low,Open,Volume
0,2024-09-16 00:00:00+00:00,10.06666279,10.08611479,9.920768574,9.920768574,4498600
1,2024-02-09 00:00:00+00:00,7.456766605,7.52152732,7.419760357,7.466017506,8509300
2,2024-02-07 00:00:00+00:00,7.549281597,7.678803905,7.530778914,7.66030034,7352300
3,2024-02-08 00:00:00+00:00,7.549281597,7.595539627,7.503023567,7.567785162,5665700
4,2024-04-16 00:00:00+00:00,7.577036381,7.651049762,7.549281914,7.651049762,5795100


In [46]:
# Add Yahoo symbol to the DataFrame
df_VOD['symbol'] = 'YAHOO'

# Preview the first few rows of the DataFrame
df_VOD.head()

,Date,Close,High,Low,Open,Volume,symbol
0,2024-09-16 00:00:00+00:00,10.06666279,10.08611479,9.920768574,9.920768574,4498600,YAHOO
1,2024-02-09 00:00:00+00:00,7.456766605,7.52152732,7.419760357,7.466017506,8509300,YAHOO
2,2024-02-07 00:00:00+00:00,7.549281597,7.678803905,7.530778914,7.66030034,7352300,YAHOO
3,2024-02-08 00:00:00+00:00,7.549281597,7.595539627,7.503023567,7.567785162,5665700,YAHOO
4,2024-04-16 00:00:00+00:00,7.577036381,7.651049762,7.549281914,7.651049762,5795100,YAHOO


In [47]:
# Make column name in lower case
df_VOD.columns = df_VOD.columns.str.lower()
df_VOD

,date,close,high,low,open,volume,symbol
0,2024-09-16 00:00:00+00:00,10.06666279,10.08611479,9.920768574,9.920768574,4498600,YAHOO
1,2024-02-09 00:00:00+00:00,7.456766605,7.52152732,7.419760357,7.466017506,8509300,YAHOO
2,2024-02-07 00:00:00+00:00,7.549281597,7.678803905,7.530778914,7.66030034,7352300,YAHOO
3,2024-02-08 00:00:00+00:00,7.549281597,7.595539627,7.503023567,7.567785162,5665700,YAHOO
4,2024-04-16 00:00:00+00:00,7.577036381,7.651049762,7.549281914,7.651049762,5795100,YAHOO
...,...,...,...,...,...,...,...
248,2024-09-13 00:00:00+00:00,9.891590118,9.92076859,9.85268518,9.85268518,4756700,YAHOO
249,2024-09-17 00:00:00+00:00,9.901316643,9.998578526,9.823506765,9.998578526,5087000,YAHOO
250,2024-09-05 00:00:00+00:00,9.911042213,10.00830502,9.901316674,9.998578558,6251800,YAHOO
251,2024-09-18 00:00:00+00:00,9.949947357,10.10556804,9.930495351,9.940221818,5187100,YAHOO


# Data Concatenation

In [48]:
import pandas as pd

# Concatenate the DataFrames, ignoring the index (so that they don't overlap)
df_combined = pd.concat([df_Yahoo, df_VOD, df_BABA, df_AAPL, df_IBM], ignore_index=True)

# Preview the combined DataFrame
df_combined.head()

,date,open,high,low,close,volume,symbol
0,2023-04-28 00:00:00+00:00,33797.43,34104.56,33728.4,34098.16,354310000,YAHOO
1,2023-04-27 00:00:00+00:00,33381.66,33859.75,33374.65,33826.16,343240000,YAHOO
2,2023-04-26 00:00:00+00:00,33596.34,33645.83,33235.85,33301.87,321170000,YAHOO
3,2023-04-25 00:00:00+00:00,33828.34,33875.49,33525.39,33530.83,297880000,YAHOO
4,2023-04-24 00:00:00+00:00,33805.04,33891.15,33726.09,33875.4,252020000,YAHOO


# Analysis

In [49]:
# Check for missing values
missing_values = df_combined.isnull().sum()

# Summary statistics for numerical columns
summary_statistics = df_combined.describe()

# Group by 'symbol' to see the distribution of data per symbol
grouped_by_symbol = df_combined.groupby('symbol').size()

# Check for duplicates
duplicates = df_combined.duplicated().sum()

# Check data types of each column
data_types = df_combined.dtypes

# Combine the results into a dictionary for better readability
analysis_results = {
    "Missing Values": missing_values,
    "Summary Statistics": summary_statistics,
    "Grouped by Symbol": grouped_by_symbol,
    "Number of Duplicates": duplicates,
    "Data Types": data_types
}

# Display the results
analysis_results

{'Missing Values': date      2
 open      1
 high      1
 low       1
 close     1
 volume    0
 symbol    0
 dtype: int64,
 'Summary Statistics':                              date     open    high     low   close     volume  \
 count                        1962  1963.00  1963.0  1963.0  1963.0       1964   
 unique                       1710  1919.00  1920.0  1922.0  1854.0       1918   
 top     2024-08-21 00:00:00+00:00   227.48   227.5   227.0   227.4  301750000   
 freq                            2    10.00    15.0    10.0     8.0          3   
 
        symbol  
 count    1964  
 unique      4  
 top     YAHOO  
 freq     1512  ,
 'Grouped by Symbol': symbol
 AAPL      252
 BABA      100
 IBM       100
 YAHOO    1512
 dtype: int64,
 'Number of Duplicates': np.int64(0),
 'Data Types': date      object
 open      object
 high      object
 low       object
 close     object
 volume    object
 symbol    object
 dtype: object}

In [50]:
# Convert 'date' column to datetime type
df_combined['date'] = pd.to_datetime(df_combined['date'], errors='coerce')

# Fill missing values for numerical columns with mean
df_combined['open'] = pd.to_numeric(df_combined['open'], errors='coerce')
df_combined['high'] = pd.to_numeric(df_combined['high'], errors='coerce')
df_combined['low'] = pd.to_numeric(df_combined['low'], errors='coerce')
df_combined['close'] = pd.to_numeric(df_combined['close'], errors='coerce')
df_combined['volume'] = pd.to_numeric(df_combined['volume'].replace({',': ''}, regex=True), errors='coerce')


In [51]:
# Fill missing numerical values with forward-fill or mean
df_combined.fillna({
    'open': df_combined['open'].mean(),
    'high': df_combined['high'].mean(),
    'low': df_combined['low'].mean(),
    'close': df_combined['close'].mean(),
    'volume': df_combined['volume'].mean()
}, inplace=True)


In [52]:

# Drop rows with missing 'date' or 'symbol' (optional, depending on context)
df_combined.dropna(subset=['date', 'symbol'], inplace=True)

# Convert 'symbol' to categorical type for better performance
df_combined['symbol'] = df_combined['symbol'].astype('category')

# Check for missing values again
missing_values = df_combined.isnull().sum()

In [53]:
missing_values

,0
date,0
open,0
high,0
low,0
close,0
volume,0
symbol,0


In [54]:
df_combined

,date,open,high,low,close,volume,symbol
0,2023-04-28 00:00:00+00:00,33797.43,34104.560,33728.4000,34098.16,354310000.0,YAHOO
1,2023-04-27 00:00:00+00:00,33381.66,33859.750,33374.6500,33826.16,343240000.0,YAHOO
2,2023-04-26 00:00:00+00:00,33596.34,33645.830,33235.8500,33301.87,321170000.0,YAHOO
3,2023-04-25 00:00:00+00:00,33828.34,33875.490,33525.3900,33530.83,297880000.0,YAHOO
4,2023-04-24 00:00:00+00:00,33805.04,33891.150,33726.0900,33875.40,252020000.0,YAHOO
...,...,...,...,...,...,...,...
1959,2025-04-04 15:55:00+00:00,234.12,234.420,233.2400,233.28,39294.0,IBM
1960,2025-04-04 15:50:00+00:00,234.23,234.595,233.6800,233.98,52635.0,IBM
1961,2025-04-04 15:45:00+00:00,233.59,234.430,233.0320,234.43,51227.0,IBM
1962,2025-04-04 15:40:00+00:00,233.60,234.130,233.2101,233.60,41911.0,IBM


In [ ]:
df_combined.to_csv("/content/drive/My Drive/ETL_Pipeline/data/final_cleaned_data.csv", index=False)


# Data Upsertion In MangoDB

In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pymongo import MongoClient

# MongoDB connection
uri = "mongodb+srv://cluster0.e9zjlwb.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&appName=Cluster0"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='/content/drive/MyDrive/ETL_Pipeline/X509-cert-2867975372075294145.pem')
client = MongoClient(uri)
db = client["Clusters"]
collection = db["stock_data"]

In [ ]:
# Convert the DataFrame to a list of dictionaries
data_dict = df_combined.to_dict(orient="records")

# Insert the data into MongoDB
collection.insert_many(data_dict)

print("Data uploaded to MongoDB successfully!")

Data uploaded to MongoDB successfully!


#Data Retrieval

In [ ]:
import pandas as pd
from pymongo import MongoClient

# MongoDB connection
uri = "mongodb+srv://cluster0.e9zjlwb.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&appName=Cluster0"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='/content/drive/MyDrive/ETL_Pipeline/X509-cert-2867975372075294145.pem')
db = client["Clusters"]
collection = db["stock_data"]

# Retrieve the data from MongoDB
data = collection.find()

# Convert the MongoDB result to a Pandas DataFrame
df_retrieved = pd.DataFrame(list(data))

# Display the first few rows of the DataFrame
df_retrieved.head()

,_id,date,open,high,low,close,volume,symbol,daily_pct_change
0,67f026151ce8915018d6ca9d,2023-04-28,33797.43,34104.56,33728.40,34098.16,354310000.0,YAHOO,NaN
1,67f026151ce8915018d6ca9e,2023-04-27,33381.66,33859.75,33374.65,33826.16,343240000.0,YAHOO,-0.797697
2,67f026151ce8915018d6ca9f,2023-04-26,33596.34,33645.83,33235.85,33301.87,321170000.0,YAHOO,-1.549954
3,67f026151ce8915018d6caa0,2023-04-25,33828.34,33875.49,33525.39,33530.83,297880000.0,YAHOO,0.687529
4,67f026151ce8915018d6caa1,2023-04-24,33805.04,33891.15,33726.09,33875.40,252020000.0,YAHOO,1.027621
